In [1]:
!python train.py

EPOCH: [1/100], TRAIN_LOSS: 0.195, VALID_LOSS: 0.185
save wieghts to ./weights/resnet50_0.18472.pth
EPOCH: [2/100], TRAIN_LOSS: 0.194, VALID_LOSS: 0.140
save wieghts to ./weights/resnet50_0.13990.pth
EPOCH: [3/100], TRAIN_LOSS: 0.190, VALID_LOSS: 0.122
save wieghts to ./weights/resnet50_0.12224.pth
EPOCH: [4/100], TRAIN_LOSS: 0.188, VALID_LOSS: 0.131
save wieghts to ./weights/resnet50_0.13143.pth
EPOCH: [5/100], TRAIN_LOSS: 0.184, VALID_LOSS: 0.144
save wieghts to ./weights/resnet50_0.14398.pth
EPOCH: [6/100], TRAIN_LOSS: 0.164, VALID_LOSS: 0.107
save wieghts to ./weights/resnet50_0.10714.pth
EPOCH: [7/100], TRAIN_LOSS: 0.152, VALID_LOSS: 0.151
save wieghts to ./weights/resnet50_0.15135.pth
EPOCH: [8/100], TRAIN_LOSS: 0.153, VALID_LOSS: 0.134
save wieghts to ./weights/resnet50_0.13416.pth
EPOCH: [9/100], TRAIN_LOSS: 0.138, VALID_LOSS: 0.093
save wieghts to ./weights/resnet50_0.09253.pth
EPOCH: [10/100], TRAIN_LOSS: 0.126, VALID_LOSS: 0.068
save wieghts to ./weights/resnet50_0.06793.pth

save wieghts to ./weights/resnet50_0.05963.pth
EPOCH: [83/100], TRAIN_LOSS: 0.001, VALID_LOSS: 0.023
save wieghts to ./weights/resnet50_0.02297.pth
EPOCH: [84/100], TRAIN_LOSS: 0.001, VALID_LOSS: 0.039
save wieghts to ./weights/resnet50_0.03886.pth
EPOCH: [85/100], TRAIN_LOSS: 0.002, VALID_LOSS: 0.072
save wieghts to ./weights/resnet50_0.07215.pth
EPOCH: [86/100], TRAIN_LOSS: 0.001, VALID_LOSS: 0.065
save wieghts to ./weights/resnet50_0.06502.pth
EPOCH: [87/100], TRAIN_LOSS: 0.001, VALID_LOSS: 0.053
save wieghts to ./weights/resnet50_0.05330.pth
EPOCH: [88/100], TRAIN_LOSS: 0.001, VALID_LOSS: 0.043
save wieghts to ./weights/resnet50_0.04303.pth
EPOCH: [89/100], TRAIN_LOSS: 0.002, VALID_LOSS: 0.047
save wieghts to ./weights/resnet50_0.04745.pth
EPOCH: [90/100], TRAIN_LOSS: 0.002, VALID_LOSS: 0.041
save wieghts to ./weights/resnet50_0.04143.pth
EPOCH: [91/100], TRAIN_LOSS: 0.001, VALID_LOSS: 0.030
save wieghts to ./weights/resnet50_0.02982.pth
EPOCH: [92/100], TRAIN_LOSS: 0.001, VALID_LO

In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
import seaborn as sns
import pandas as pd

from datasets import TripletSampler, load_datasets, load_test
from models import TripletResNet
from losses import TripletLoss, TripletAngularLoss
from params import args


transform = transforms.Compose([transforms.Resize(args.img_size),
                                transforms.CenterCrop(args.img_size),
                                transforms.ToTensor()])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

test_dataset = load_test(args.test_json, transform)
test_loader = DataLoader(test_dataset, batch_size=126)

model = TripletResNet(args.output_dim)
model = model.to(device)
# model.load_state_dict(torch.load('./weights/' + args.experiment_name +'_features'+ str(args.output_dim) + '_Ture.pth'))  #
model.load_state_dict(torch.load('./weights/resnet50_0.01347.pth')) #_features256_True
model.eval()
# model = torchvision.models.__dict__['resnet50'](pretrained=True)
# model = model.to(device)
# model.eval()

# AA = model.model[-1]
# handle_forward = model.model.register_forward_hook(hook)
# handle_backward = model.model.register_backward_hook(hook)
for param in model.parameters():
    param.requires_grad = True
pred_metric = []
y_test = []
for i, (anchors, positives, negatives, labels) in enumerate(test_loader):
    anchors = anchors.to(device)
    positives = positives.to(device)
    negatives = negatives.to(device)
    metric = model(anchors)
    # metric_positives = model(positives)
    # metric_negatives = model(negatives)

    pred_metric.append(metric.detach().cpu().numpy())
    y_test.append(labels.detach().numpy())

pred_metric = np.concatenate(pred_metric, 0)
y_test = np.concatenate(y_test, 0)


In [2]:
############################################################
# Plot t-SNE
############################################################
y_reduced = TSNE(n_components=2, random_state=0).fit_transform(pred_metric)
plt_tSNE = plt
plt_tSNE.scatter(y_reduced[y_test == 0, 0], y_reduced[y_test == 0, 1], color='blue')
plt_tSNE.scatter(y_reduced[y_test == 1, 0], y_reduced[y_test == 1, 1], color='red')
plt_tSNE.scatter(y_reduced[y_test == 2, 0], y_reduced[y_test == 2, 1], color='green')
# plt_tSNE.scatter(y_reduced[y_test == 3, 0], y_reduced[y_test == 3, 1], color='black')

plt_tSNE.legend(['Covid19 Infected', 'Healthy', 'Tuberculosis'], loc='upper left')
plt_tSNE.savefig(args.experiment_name + '_tSNE.png')
plt_tSNE.show()

/mnt/appl/software/matplotlib/3.0.3-fosscuda-2019a-Python-3.7.2/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [3]:
############################################################
# Plot Comparing Result
############################################################
n_queries = 3
xp_idx0 = np.random.choice(np.where(labels == 0)[0])
xp_idx1 = np.random.choice(np.where(labels == 1)[0])
xp_idx2 = np.random.choice(np.where(labels == 2)[0])
# xp_idx3 = np.random.choice(np.where(labels == 3)[0])

# queries = [xp_idx0, xp_idx1, xp_idx2, xp_idx3]
queries = [xp_idx0, xp_idx1, xp_idx2]
queries_metric = metric[queries, :]


euc_dist = torch.cdist(queries_metric, metric, p=2)
cloeset_50_value, cloeset_50_pos = torch.topk(euc_dist, 42, dim=1, largest=False, sorted=True, out=None)
cloeset_50_pos = cloeset_50_pos.cpu()

##################### Visualise the gradients map w.r.t raw image
plt_Compare = plt
plt_Compare.clf()
f, axarr = plt_Compare.subplots(n_queries, 1, gridspec_kw={'wspace': 0, 'hspace': 0})
for i in range(n_queries):
    ax = axarr[i]
    # img_t = torch.cat((anchors[queries[i], :, :, :].unsqueeze(0), anchors[cloeset_50_pos[i, :], :, :, :]))
    img_t = anchors[cloeset_50_pos[i, 0:5], :, :, :]
    plot_out = torchvision.utils.make_grid(img_t, nrow=5, normalize=True, padding=1)
    plot_out = plot_out.cpu()
    plot_out.size()
    image_transposed = plot_out.permute([1, 2, 0])
    ax.imshow(image_transposed)
    ax.axis('off')
plt_Compare.savefig(args.experiment_name + '_Comparring.png')
f.show()

In [6]:
############################################################
# Calculating mAP
############################################################
def th_delete(tensor, indices):
    mask = torch.ones(tensor.numel(), dtype=torch.bool)
    mask[indices] = False
    return tensor[mask]

predict_label = torch.cat(([labels[cloeset_50_pos[0, 1:]], labels[cloeset_50_pos[1, 1:]], labels[cloeset_50_pos[2, 1:]]]), 0)
labels = th_delete(labels, queries)
torch.sum(predict_label == labels)
accuracy_rate = torch.sum(predict_label == labels)/len(labels)
print(accuracy_rate)

tensor(0.9837)


In [7]:
 ############################################################
# Plot Confusion Matrix
############################################################
from sklearn.metrics import confusion_matrix

# classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# classes = ['CAP', 'Covid', 'HC', 'SPT']
classes = ['COVID', 'NORMAL', 'TUBER']

cm = confusion_matrix(labels, predict_label, labels=None, sample_weight=None)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm = np.around(cm, decimals=2)
f = plt.figure(figsize=(15, 10), dpi=300)
ax = plt.subplot(111)
df = pd.DataFrame(cm, index=classes, columns=classes)
sns.heatmap(df, cmap='Blues', annot=True, fmt='g')
ax.xaxis.tick_top()
ax.set_ylabel('True', fontsize=15)
ax.set_xlabel('Pred', fontsize=15)
ax.tick_params(axis='y', labelsize=15, labelrotation=45)  # y axis
ax.tick_params(axis='x', labelsize=15)  # x axis
f.savefig('%s.jpg' % 'Confusion_Matrix', bbox_inches='tight')
f.show()


/mnt/appl/software/matplotlib/3.0.3-fosscuda-2019a-Python-3.7.2/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [8]:
############################################################
from sklearn.metrics import f1_score

ma_f1 = f1_score(labels, predict_label, average='macro')
mi_f1 = f1_score(labels, predict_label, average='micro')
print(ma_f1, mi_f1)

0.9837301587301588 0.983739837398374


In [10]:
 ############################################################
# Plot ROC curve, and PR curve
############################################################
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

num_classes = 3
# scores = torch.softmax(output, dim=1).detach().numpy()
scores = label_binarize(predict_label, classes=list(range(num_classes))) # out = model(data)
binary_label = label_binarize(labels, classes=list(range(num_classes)))  # num_classes=10

fpr = {}
tpr = {}
roc_auc = {}

for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(binary_label[:, i], scores[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(binary_label.ravel(), scores.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))
# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(num_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
# Finally average it and compute AUC
mean_tpr /= num_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

plt.figure(figsize=(8, 8))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

for i in range(num_classes):
    plt.plot(fpr[i], tpr[i], lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.grid()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC')
plt.legend(loc="lower right")
plt.savefig('Multi-class ROC.jpg', bbox_inches='tight')
plt.show()

/mnt/appl/software/matplotlib/3.0.3-fosscuda-2019a-Python-3.7.2/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [6]:
############################################################

predict_label = torch.reshape(predict_label, (4, 27))
labels = torch.reshape(labels, (4, 27))

TP0 = torch.sum(predict_label[0, :] == labels[0, :])/len(labels[0, :])
FP0 = torch.sum(predict_label[0, :] != labels[0, :])/len(labels[0, :])
FN0 = torch.sum(predict_label[0, :] != labels[0, :])/len(labels[0, :])

TP1 = torch.sum(predict_label[1, :] == labels[1, :])/len(labels[1, :])
FP1 = torch.sum(predict_label[1, :] != labels[1, :])/len(labels[1, :])
FN1 = torch.sum(predict_label[1, :] != labels[1, :])/len(labels[1, :])

TP2 = torch.sum(predict_label[2, :] == labels[1, :])/len(labels[2, :])
FP2 = torch.sum(predict_label[2, :] != labels[1, :])/len(labels[2, :])
FN2 = torch.sum(predict_label[2, :] != labels[1, :])/len(labels[2, :])


accuracy_rate_0 = torch.sum(predict_label[0, :] == labels[0, :])/len(labels[0, :])
accuracy_rate_1 = torch.sum(predict_label[1, :] == labels[1, :])/len(labels[1, :])
accuracy_rate_2 = torch.sum(predict_label[2, :] == labels[2, :])/len(labels[2, :])
# accuracy_rate_3 = torch.sum(predict_label[3, :] == labels[3, :])/len(labels[3, :])

from sklearn import metrics
aps = []
for y_t, y_s in zip(labels, predict_label):
    ap = metrics.average_precision_score(y_t, y_s)
    aps.append(ap)
np.mean(np.array(aps))

ha = 1

tensor([[ 24,  32,  42,  28,  30,  78,  48,  13,  73,  57,  75,  18,  50,  54,
          26,   5,  65,  58,  14,  76,   1,  44,  10,   2,  70,  52,  21,  83],
        [ 81,  43,  79,  46,  68,  58,  51,  54,  80,  55,  61,  67,  52,  62,
          59,  64,  49,  66,  57,  50,  74,  60,  72,  71,  47,  82,  44,  56],
        [ 96, 116,  86,  85,  95, 113, 103, 101,  97, 107,  89,  87, 119, 125,
         117, 123, 109,  98,  90, 100, 108, 120, 110,  94,  91, 115, 105,  93]],
       device='cuda:0')